<a href="https://colab.research.google.com/github/kevinrvb16/Frontend-Carbon/blob/main/FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
#Este é um código super básico que extrai filmes da página principal do NDMais Florianopolis
#Foi feito no Colab

# -*- coding: utf-8 -*-
#"""WebScraping-superBasico.ipynb

#Automatically generated by Colaboratory.

import requests # para requisições http
import json # para gerar JSON a partir de objetos do Python
from bs4 import BeautifulSoup # BeautifulSoup é uma biblioteca Python de extração de dados de arquivos HTML e XML.
from google.colab import files #importei esse pacote para fazer o download do arquivo JSON gerado

sites = {
  "rottentomatoes.com":
    {
      "tagFilmes": "tiles-carousel-responsive-item-deprecated",
      "titulo": {
          "tag": "span",
          "classe": "p--small"
      },
      "criticScore": {
          "tag": "rt-text",
          "slot": "criticsScore"
      },
      "audienceScore": {
          "tag": "rt-text",
          "slot": "audienceScore"
      },
      "link": {
          "tag": "a",
          "data-track": "scores"
      }
    },
  "rogerebert.com/reviews":
    {
      "tagFilmes": "article",
      "titulo": {
          "tag": "h3",
          "classe": "text-2xl"
      },
      "critico": {
          "tag": "div",
          "classe": "montserrat-500 mt-2 text-meta-grey text-sm"
      },
      "link": {
          "tag": "a",
          "classe": "image-hover cursor-pointer relative rounded flex flex-col justify-end"
      },
      "criticScore": {
          "tag": "img",
          "classe": "filled"
      }
    },
  "adorocinema.com/filmes/criticas-filmes":
    {
      "tagFilmes": "li",
      "titulo": {
          "tag": "a",
          "classe": "meta-title-link"
      },
      "criticScore": {
          "tag": "span",
          "classe": "stareval-note"
      },
      "sinopse": {
          "tag": "div",
          "classe": "content-txt"
      },
      "link": {
          "tag": "a",
          "classe": "meta-title-link"
      }
    }
}
#cria uma variável para numerar as filmes
filmes_nr = 1
# #cria uma variável do tipo lista para guardar os dados em um JSON
resposta = []
replacers = [ "\n\t\t\t", "\n                    ", "\n        \n            ", "\t\t", "\n"]
for key, site in sites.items():
  baseURL = "https://www." + key
  requisicaoDePagina = requests.get(baseURL)

  conteudo = requisicaoDePagina.content

  #mostra o tipo Pyhton da página
  # print(type(requisicaoDePagina.content))

  #joga para a variável site todo o conteúdo da página passada pelo requests.get()
  siteDOM = BeautifulSoup(conteudo, 'html.parser')

  #imprime o site inteiro, como o original
  # print(siteDOM)
  print("-----------------")

  print(site)
  #joga para a variável filmes todos os elementos "article", que é onde está cada uma das manchetes do site princial
  if site["tagFilmes"] == "li":
    filmes = siteDOM.findAll("li", {"class" : "hred"})
  else:
    filmes = siteDOM.findAll(site["tagFilmes"])

  # #imprime tipo Python de filmes
  print(type(filmes))

  # #faz um laço na lista filmes (no plural), atribuindo cada item da lista para a variável noticia (no singular)
  for filme in filmes:

  #   #encontra nas tags do HTMO título, resumo e onde está publicada cada uma das filmes, e joga para as respectivas variáveis
  #titulo = filme.find("span", {"class" : "p--small"})
    titulo = filme.find(site["titulo"]["tag"], {"class" : site["titulo"]["classe"]})
    if site["criticScore"]["tag"] == "rt-text":
      criticScore = filme.find(site["criticScore"]["tag"], {"slot" : site["criticScore"]["slot"]})
    else:
      criticScore = filme.find(site["criticScore"]["tag"], {"class" : site["criticScore"]["classe"]})

    if "audienceScore" in site:
      audScore = filme.find("rt-text", {"slot" : "audienceScore"})
    elif "critico" in site:
      crt = filme.find("div", {"class" : "montserrat-500 mt-2 text-meta-grey text-sm"})
    elif "sinopse" in site:
      snps = filme.find("div", {"class" : "content-txt"})

    if "data-track" in site["link"]:
      link = filme.find(site["link"]["tag"], {"data-track" : site["link"]["data-track"]})
    else:
      link = filme.find(site["link"]["tag"], {"class" : site["link"]["classe"]})

    baseURL
    if link:
      lnk = baseURL + link.get('href')
    elif baseURL == "https://www.rottentomatoes.com":
      link = filme.find("a")
      lnk = baseURL + link.get('href')
    else:
      lnk = ""

  #joga o texto de cada uma das tags
    tit = titulo.text

    if baseURL == "https://www.rogerebert.com/reviews":
      lastClass = criticScore['class'][2]
      criticScore = str(float(lastClass.split('star')[1]) * 2.5) + '%'
    elif baseURL == "https://www.adorocinema.com/filmes/criticas-filmes":
      stars = criticScore.text
      rate = stars.replace(',', '')
      value = float(rate) * 2
      criticScore = str(value) + '%'
    else:
      critSco = criticScore.text
    critc = ""
    snpse = ""
    audSco = ""
    if "audienceScore" in site:
      audSco = audScore.text
    elif "critico" in site:
      critc = crt.text
    elif "sinopse" in site:
      snpse = snps.text

  # #como pode não haver resumo em alguns filmes, é feito um teste.
  #   if link:
  #     lnk = "https://www.rottentomatoes.com"+ link.get('href')
  #   else:
  #     link = filme.find("a")
  #     lnk = "https://www.rottentomatoes.com" + link.get('href')

  #   print("Título:", tit)
  #   print("Critic Score:", critSco)
  #   print("Audience Score:", audSco)
  #   print("Link:", lnk)


  #   #um print para separa os filmes
    print("....")


  #   # Cria uma espécie de dicionario para depois jogar para o JSON
    for replacer in replacers:
      tit = tit.replace(replacer, "")
      critc = critc.replace(replacer, "")
      snpse = snpse.replace(replacer, "")

    dados = {'NUMERO': str(filmes_nr), 'TITULO': tit, 'CRITIC SCORE': critSco, 'AUDIENCE SCORE': audSco, 'CRITICO': critc, 'SINOPSE': snpse, 'LINK': lnk}


  #   # Pendura o dicionario em uma lista e incrementa a variável que conta o número de filmes
    resposta.append(dados)
    filmes_nr += 1

  # #final do laço que percorre a lista de notícias

# #apenas dois prints para mostrar o tipo da resposta e a resposta transformada em string
print("Tipo da resposta", type(resposta))
print(' '.join(map(str, resposta)))


# # Converte os objetos Pyhton em objeto JSON e exporta para o filmes.json
with open('filmes.json', 'w') as arquivo:
  arquivo.write(str(json.dumps(resposta, indent=4)))
print("Created Json File")

# #faz o download usando a boblioteca do Colab
files.download('filmes.json')




-----------------
{'tagFilmes': 'tiles-carousel-responsive-item-deprecated', 'titulo': {'tag': 'span', 'classe': 'p--small'}, 'criticScore': {'tag': 'rt-text', 'slot': 'criticsScore'}, 'audienceScore': {'tag': 'rt-text', 'slot': 'audienceScore'}, 'link': {'tag': 'a', 'data-track': 'scores'}}
<class 'bs4.element.ResultSet'>
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
-----------------
{'tagFilmes': 'article', 'titulo': {'tag': 'h3', 'classe': 'text-2xl'}, 'critico': {'tag': 'div', 'classe':

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>